# Cleaning and batch-processing data 🧽

**Author:** James Cranley (<james.cranley@doctors.org.uk>)  
**Date:** March 2025

---

## Overview

This notebook shows how programming approaches can save a lot of time compared to manual approaches to clean data.

### What You'll Learn

- **Reading in multiple files**
- **Regular expressions:** Use regex to pattern-match
- **Batch processing:** Apply a standardised edit to multiple files

## Exercise
<div style="border-left: 4px solid #007acc; padding: 0.5em 1em; background-color: #f0f8ff; margin: 1em 0;">
  <h4>👨‍🏫 Scenario</h4>
  <p>Bob here, I've send you some HbA1c data collected over previous years, can you clean it up and send it back to me?</p>
</div>

In [1]:
import os # operating system: for interacting with files
import pandas # key tabular data package in python
import numpy # important package for mathematics in python
import re # the regualr expression package, used for pattern-matching, see below
import time  # the time module, useful for...timing things

### Reading data into dataframes

In [2]:
input_directory_path = "./data/hba1c/" # this is the path to a folder

We us the os module to inspect what is in the directory

In [3]:
os.listdir(input_directory_path)

['2021.xlsx',
 '2017.xlsx',
 '2016.xlsx',
 '2020.xlsx',
 '2024.xlsx',
 '2019.xlsx',
 '2023.xlsx',
 '2015.xlsx',
 '2014.xlsx',
 '2022.xlsx',
 '2018.xlsx']

A bunch of Excel files, one per year

### Inspect the data

In [4]:
sample_file_1 = "2021.xlsx" # let's choose one file

In [5]:
sample_file_1_path = os.path.join(input_directory_path,sample_file_1) # this line creates a path to that file
sample_file_1_path

'./data/hba1c/2021.xlsx'

As we did before, we use pandas to read in the file

In [6]:
sample_dataframe_1 = pandas.read_excel(sample_file_1_path)
sample_dataframe_1

,patient ID,hba1c_mmol
0,HOSP557373,44
1,HOSP470321,50
2,HOSP112289,40
3,HOSP800336,41
4,HOSP798956,40
...,...,...
95,HOSP357681,49
96,HOSP194018,43
97,HOSP486601,41
98,HOSP394054,40


OK, we can see two columns, patient ID and a column for HbA1c.

Let's inspect another file...

In [7]:
sample_file_2 = "2016.xlsx" # let's choose one file

In [8]:
sample_file_2_path = os.path.join(input_directory_path,sample_file_2) # this line creates a path to that file
sample_file_2_path

'./data/hba1c/2016.xlsx'

As we did before, we use pandas to read in the file

In [9]:
sample_dataframe_2 = pandas.read_excel(sample_file_2_path)
sample_dataframe_2

,ID #,HBA1C
0,HOSP350875,6.2
1,HOSP563389,6.1
2,HOSP170390,7.2
3,HOSP807689,6.4
4,HOSP353490,5.9
...,...,...
95,HOSP772108,6.7
96,HOSP467257,6.1
97,HOSP366765,5.9
98,HOSP246106,6.3


🚨 This file also has 2 columns (for ID and HbA1c), but, there are problems...

1) The **column names are not consistent**
2) The HbA1c **units are not consistent**

Let's **make them uniform**

In [10]:
excel_files = os.listdir(input_directory_path) # We make a list of the excel files
excel_files

['2021.xlsx',
 '2017.xlsx',
 '2016.xlsx',
 '2020.xlsx',
 '2024.xlsx',
 '2019.xlsx',
 '2023.xlsx',
 '2015.xlsx',
 '2014.xlsx',
 '2022.xlsx',
 '2018.xlsx']

Now let's go through the list, read each excel file in and check the column names

For this we can use a **for loop**, a way of iterating through a list.

Here is a basic for loop:

In [11]:
excel_files

['2021.xlsx',
 '2017.xlsx',
 '2016.xlsx',
 '2020.xlsx',
 '2024.xlsx',
 '2019.xlsx',
 '2023.xlsx',
 '2015.xlsx',
 '2014.xlsx',
 '2022.xlsx',
 '2018.xlsx']

In [12]:
for excel_file in excel_files:
    print(excel_file)

2021.xlsx
2017.xlsx
2016.xlsx
2020.xlsx
2024.xlsx
2019.xlsx
2023.xlsx
2015.xlsx
2014.xlsx
2022.xlsx
2018.xlsx


Now let's us the for loop to read in each file in turn and tell us the column names used

In [13]:
for excel_file in excel_files:
    print(f"Processing {excel_file}")
    file_path = os.path.join(input_directory_path,excel_file) # defines the path to that file
    dataframe = pandas.read_excel(file_path) # reads in that file
    print(dataframe.columns.tolist())

Processing 2021.xlsx
['patient ID', 'hba1c_mmol']
Processing 2017.xlsx
['PatientId', 'HbA1C %']
Processing 2016.xlsx
['ID #', 'HBA1C']
Processing 2020.xlsx
['patient_ID', 'HbA1c (mmol/mol)']
Processing 2024.xlsx
['PATIENT_ID', 'HbA1c_mmol_per_mol']
Processing 2019.xlsx
['patient_ID', 'HbA1c']
Processing 2023.xlsx
['PatientId', 'HbA1C mmol']
Processing 2015.xlsx
['patient ID', 'hba1c']
Processing 2014.xlsx
['patient_ID', 'HbA1c']
Processing 2022.xlsx
['ID #', 'HBA1C_new']
Processing 2018.xlsx
['PATIENT_ID', 'HbA1c_old']


## 🧪 Using **Pattern Matching** to handle inconsistent column names

Over the years, many different column names have been used for the **patient ID** and **HbA1c** values — but **they all share some common patterns**.

- The **patient ID column** always contains the text `"ID"` (in some form, like `patient_ID`, `ID #`, or `PatientId`)
- The **HbA1c column** always contains the text `"HbA1c"` (in some form, like `HBA1C`, `hba1c`, or `HbA1c (mmol/mol)`)

We can use **pattern matching** with something called **regular expressions (regex)** to identify these columns — even if the exact names are different.

Once we've matched them, we can **rename them** to standard names (like `patient_ID` and `HbA1c`) to make our analysis easier.

First, let's try it on a single file:

In [14]:
sample_dataframe_1 = pandas.read_excel(sample_file_1_path)
sample_dataframe_1

,patient ID,hba1c_mmol
0,HOSP557373,44
1,HOSP470321,50
2,HOSP112289,40
3,HOSP800336,41
4,HOSP798956,40
...,...,...
95,HOSP357681,49
96,HOSP194018,43
97,HOSP486601,41
98,HOSP394054,40


We use a for loop to loop through each column (only 2 of course!) to find the column which matches the pattern we are looking for

In [15]:
# Loop through all the column names in the DataFrame
for col in sample_dataframe_1.columns:
    
    # Check if the column name contains the word "ID" (case-insensitive)
    if re.search(r'ID', col, re.IGNORECASE): # the \b is a word boundary
        
        # If it matches, print out the column name
        print(f"The column '{col}' matches the ID column")

The column 'patient ID' matches the ID column


Now we can extend this to **batch-process all the files:** identify the ID and HbA1c columns, rename them, and save the cleaned table

Let's define and make a results folder

In [16]:
output_directory_path = './results/hba1c_cleaned'
os.makedirs(output_directory_path, exist_ok=True)

We now loop through each file, read it in, search for the ID column and rename it, do the same for the HbA1c column and then save the renamed files

In [17]:
for excel_file in excel_files:
    print(f"Processing {excel_file}...")
    file_path = os.path.join(input_directory_path, excel_file)
    df = pandas.read_excel(file_path)

    # Identify and rename the ID column
    for col in df.columns:
        if re.search(r'\bID\b', col, re.IGNORECASE):
            df.rename(columns={col: 'patient_ID'}, inplace=True)

    # Identify and rename the HbA1c column
    for col in df.columns:
        if re.search(r'HBA1C', col, re.IGNORECASE):
            df.rename(columns={col: 'HbA1c'}, inplace=True)

    # Save cleaned file with the same filename in the new directory
    output_file_path = os.path.join(output_directory_path, excel_file)
    df.to_excel(output_file_path, index=False)

Processing 2021.xlsx...
Processing 2017.xlsx...
Processing 2016.xlsx...
Processing 2020.xlsx...
Processing 2024.xlsx...
Processing 2019.xlsx...
Processing 2023.xlsx...
Processing 2015.xlsx...
Processing 2014.xlsx...
Processing 2022.xlsx...
Processing 2018.xlsx...


## 🎯 Batch processing to standardise HbA1c Units

You may have noticed that the units of HbA1c vary between files:

- Some files report values in **% (percent)**.
- Others report values in **mmol/mol** — this is the modern format.

To compare results properly, we need to **convert everything to the same unit**:  
✅ **mmol/mol**

We'll do this in two steps:

1. **Detect** what units the HbA1c values are likely in.
2. **Convert** the values to mmol/mol **if they are in %**.

In [18]:
sample_dataframe_1.head()

,patient ID,hba1c_mmol
0,HOSP557373,44
1,HOSP470321,50
2,HOSP112289,40
3,HOSP800336,41
4,HOSP798956,40


In [19]:
# Try to find the HbA1c column
for col in sample_dataframe_1.columns:
    if re.search(r'HBA1C', col, re.IGNORECASE):
        hba1c_col = col
        break

# Estimate unit based on typical value range
if sample_dataframe_1[hba1c_col].mean() < 20:
    print("Units appear to be in % — CONVERSION IS NEEDED.")
else:
    print("Units appear to be in mmol/mol — NO CONVERSION NEEDED.")

Units appear to be in mmol/mol — NO CONVERSION NEEDED.


In [20]:
sample_dataframe_2.head()

,ID #,HBA1C
0,HOSP350875,6.2
1,HOSP563389,6.1
2,HOSP170390,7.2
3,HOSP807689,6.4
4,HOSP353490,5.9


In [21]:
# Try to find the HbA1c column
for col in sample_dataframe_2.columns:
    if re.search(r'HBA1C', col, re.IGNORECASE):
        hba1c_col = col
        break

# Estimate unit based on typical value range
if sample_dataframe_2[hba1c_col].mean() < 20:
    print("Units appear to be in % — CONVERSION IS NEEDED.")
else:
    print("Units appear to be in mmol/mol — NO CONVERSION NEEDED.")

Units appear to be in % — CONVERSION IS NEEDED.


In [22]:
# Only convert if values are in %
if sample_dataframe_2[hba1c_col].mean() < 20:
    sample_dataframe_2[hba1c_col] = ((sample_dataframe_2[hba1c_col] - 2.15) * 10.929).round().astype(int)
    print("Converted HbA1c values from % to mmol/mol.")

sample_dataframe_2

Converted HbA1c values from % to mmol/mol.


,ID #,HBA1C
0,HOSP350875,44
1,HOSP563389,43
2,HOSP170390,55
3,HOSP807689,46
4,HOSP353490,41
...,...,...
95,HOSP772108,50
96,HOSP467257,43
97,HOSP366765,41
98,HOSP246106,45


Ok the code seems to work, so let's **combine the column renaming and unit conversion in a single loop**, and apply it to all the Excel files

In [23]:
input_directory_path = "./data/hba1c/"

output_directory_path = "./results/hba1c_cleaned_converted/"
os.makedirs(output_directory_path, exist_ok=True)

In [24]:
# Get a list of all Excel files in the input folder
excel_files = os.listdir(input_directory_path)
excel_files

['2021.xlsx',
 '2017.xlsx',
 '2016.xlsx',
 '2020.xlsx',
 '2024.xlsx',
 '2019.xlsx',
 '2023.xlsx',
 '2015.xlsx',
 '2014.xlsx',
 '2022.xlsx',
 '2018.xlsx']

In [25]:
# Start the timer before the loop
start_time = time.time()

# Loop through each file
for excel_file in excel_files:
    print(f"\n📄 Processing file: {excel_file}")
    
    # Load the Excel file into a DataFrame
    file_path = os.path.join(input_directory_path, excel_file)
    df = pandas.read_excel(file_path)
    
    # Step 1: Find the HbA1c column using regex
    hba1c_col = None
    for col in df.columns:
        if re.search(r'HBA1C', col, re.IGNORECASE):
            hba1c_col = col
            break
    
    # Step 2: Estimate the units based on average value
    if df[hba1c_col].mean() < 20:
        # If values are likely in %, convert to mmol/mol
        df[hba1c_col] = ((df[hba1c_col] - 2.15) * 10.929).round().astype(int)
        print("🔄 Converted HbA1c values from % to mmol/mol.")
    else:
        print("✅ HbA1c values already in mmol/mol. No conversion needed.")
    
    # Step 3: Rename the HbA1c column to a consistent name
    df.rename(columns={hba1c_col: 'HbA1c'}, inplace=True)

    # Step 4: Find the patient ID column and rename it
    id_col = None
    for col in df.columns:
        if re.search(r'ID', col, re.IGNORECASE):
            id_col = col
            df.rename(columns={col: 'patient_ID'}, inplace=True)
            break
    
    # Step 5: Save the cleaned DataFrame to the output folder
    output_path = os.path.join(output_directory_path, excel_file)
    df.to_excel(output_path, index=False)

# Stop the timer after the loop
end_time = time.time()
elapsed_time = round(end_time - start_time, 2)
print(f"\n⏱️ Finished processing {len(excel_files)} files in {elapsed_time} seconds.")


📄 Processing file: 2021.xlsx
✅ HbA1c values already in mmol/mol. No conversion needed.

📄 Processing file: 2017.xlsx
🔄 Converted HbA1c values from % to mmol/mol.

📄 Processing file: 2016.xlsx
🔄 Converted HbA1c values from % to mmol/mol.

📄 Processing file: 2020.xlsx
✅ HbA1c values already in mmol/mol. No conversion needed.

📄 Processing file: 2024.xlsx
✅ HbA1c values already in mmol/mol. No conversion needed.

📄 Processing file: 2019.xlsx
🔄 Converted HbA1c values from % to mmol/mol.

📄 Processing file: 2023.xlsx
✅ HbA1c values already in mmol/mol. No conversion needed.

📄 Processing file: 2015.xlsx
🔄 Converted HbA1c values from % to mmol/mol.

📄 Processing file: 2014.xlsx
🔄 Converted HbA1c values from % to mmol/mol.

📄 Processing file: 2022.xlsx
✅ HbA1c values already in mmol/mol. No conversion needed.

📄 Processing file: 2018.xlsx
🔄 Converted HbA1c values from % to mmol/mol.

⏱️ Finished processing 11 files in 0.09 seconds.


This **took less than 1/10 of 1 second**. This would have taken a long time manually...

## ⛓️ Concatenating data

<div style="border-left: 4px solid #007acc; padding: 0.5em 1em; background-color: #f0f8ff; margin: 1em 0;">
  <h4>👨‍🏫 Scenario</h4>
  <p>There are lots of Excel files. Can you send me all the data in one file?</p>
</div>

In [26]:
# Set input and output paths
input_directory_path = "./results/hba1c_cleaned_converted/"

output_directory_path = "./results/hba1c_cleaned_converted_concatenated/"
os.makedirs(output_directory_path, exist_ok=True)  # Create folder if it doesn't exist

In [27]:
# Get all Excel files in the input folder
excel_files = os.listdir(input_directory_path)
excel_files

['2021.xlsx',
 '2017.xlsx',
 '2016.xlsx',
 '2020.xlsx',
 '2024.xlsx',
 '2019.xlsx',
 '2023.xlsx',
 '2015.xlsx',
 '2014.xlsx',
 '2022.xlsx',
 '2018.xlsx']

In [28]:
# Create an empty list to store each DataFrame
all_dataframes = []

# Start the timer before the loop
start_time = time.time()

# Loop through each file and read it into a DataFrame
for excel_file in excel_files:
    year = excel_file.split(".")[0] # extract the year from the filename
    file_path = os.path.join(input_directory_path, excel_file)
    df = pandas.read_excel(file_path)
    df['year'] = year # add the year to each dataframe
    all_dataframes.append(df)  # Add the DataFrame to our list

# Combine all DataFrames into one big DataFrame
combined_df = pandas.concat(all_dataframes, ignore_index=True) # Concatenate
print(f"\n📊 Combined DataFrame has {len(combined_df)} rows total")

# Save the combined DataFrame as a new Excel file
output_file_path = os.path.join(output_directory_path, "hba1c_results.xlsx")
combined_df.to_excel(output_file_path, index=False)
print(f"\n💾 Combined file saved to: {output_file_path}")

# Stop the timer after the loop
end_time = time.time()
elapsed_time = round(end_time - start_time, 2)
print(f"\n⏱️ Finished processing {len(excel_files)} files in {elapsed_time} seconds.")


📊 Combined DataFrame has 1100 rows total

💾 Combined file saved to: ./results/hba1c_cleaned_converted_concatenated/hba1c_results.xlsx

⏱️ Finished processing 11 files in 0.07 seconds.
